# Init

In [1]:
# # if used pip install package
# !pip install xgboost
# !pip install lightgbm
# !pip install wget
# !pip install gensim
# !pip install catboost
# !pip install cython

In [2]:
from __future__ import print_function
import os
import sys
import logging
import gc
import wget
import time
import tarfile
import zipfile
import functools
import random
import copy
import pickle
from tqdm import tqdm_notebook, tqdm
from itertools import product, combinations
from scipy.special import comb, perm

import scipy
import numpy as np
import pandas as pd
import catboost as cbt
from glove import *

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.linear_model import LogisticRegression,BayesianRidge,SGDClassifier,PassiveAggressiveClassifier,RidgeClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC,NuSVC,SVC
from sklearn.metrics import f1_score, roc_auc_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.text import *
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D, LSTM, GRU  #, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import concatenate
from keras.callbacks import *
from keras.optimizers import *
from keras.engine.topology import Layer
from keras.layers import *
from keras.utils import np_utils

import gensim
import xgboost as xgb
import lightgbm as lgb

from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Using TensorFlow backend.


In [3]:
# import ray
# ray.init(object_store_memory=int(100e6))
# import modin.pandas as pd
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
# os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

In [4]:
class Logger(object):
    def __init__(self, filename='default.log', stream=sys.stdout):
        self.terminal = stream
        self.log = open(filename, 'a')
        
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
        
    def flush(self):
        pass

In [5]:
# sys.stdout = Logger("logs/default.log", sys.stdout)
# sys.stderr = Logger("logs/default_err.log", sys.stderr)

In [6]:
DDIR = "data"
UDDIR = "user_data"
UFEDIR = "user_data/feat_data_v05"
UMDIR = "user_data/model_data"
RESDIR = "prediction_result"

In [7]:
UID = "user_id"

# Load data (Only once)

In [8]:
# train_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/train_preliminary.zip", out=DDIR)
# test_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/test.zip", out=DDIR)

In [9]:
# def myunzip(filename):
#     zFile = zipfile.ZipFile(filename, "r")
#     for fileM in zFile.namelist(): 
#         zFile.extract(fileM, DDIR)
#         print(fileM)
#     zFile.close()

In [10]:
# myunzip(train_fname)
# myunzip(test_fname)

# Utils

In [11]:
def bch_rencol(values, prefix="", suffix=""):
    return list(map(lambda x: f"{prefix}"+"_".join(list(map(lambda y: str(y), x)))+f"{suffix}" 
                    if hasattr(x, "__iter__") and not isinstance(x, str) 
                    else f"{prefix}"+str(x)+f"{suffix}", values))

In [12]:
def mynunique(values):
    return values.nunique(dropna=False)
def getidxmax(x):
    return x.idxmax()[1]
# for time series
def at_len(x):
    return len(x)

def at_sum(x):
    return np.sum(x)

def at_max(x):
    return np.max(x)

def at_min(x):
    return np.min(x)

def at_mean(x):
    return np.mean(x)

def at_range(x):
    return at_max(x) - at_min(x)

def at_nunq(x):
    return len(set(x))

def at_lenDrange(x):
    return at_len(x)/(at_range(x)+1)

def at_lenDnunq(x):
    return at_len(x)/at_nunq(x)

def at_percentile(n):
    def at_percentile_(x):
        return np.percentile(x, n)
    at_percentile_.__name__ = f"at_percentile_{n}"
    return at_percentile_

In [13]:
OP_SET = ["sum", "max", "min", "mean", "std"]
OP_SET1 = ["nunique", "sum", "max", "min", "mean", "std", "median", "skew", at_percentile(0.25), at_percentile(0.75)]
OP_SET2 = ["sum", "max", "min", "mean", "std", "median", "skew", at_percentile(0.25), at_percentile(0.75)]

In [14]:
ID_SET = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry", "time", "click_times"]

In [15]:
nesting_level = 0
is_start = None
class Timer:
    def __init__(self):
        self.start = time.time()
        self.history = [self.start]

    def check(self, info):
        current = time.time()
        print(f"[{info}] spend {current - self.history[-1]:0.2f} sec")
        self.history.append(current)

def log(entry):
    global nesting_level
    space = "-" * (4 * nesting_level)
    print(f"{space}{entry}")

def timeit(method, start_log=None):
    @functools.wraps(method)
    def timed(*args, **kw):
        global is_start
        global nesting_level

        if not is_start:
            print()

        is_start = True
        log(f"Start [{method.__name__}]:" + (start_log if start_log else ""))
        log(f'Start time: {time.strftime("%Y-%m-%d %H:%M:%S")}')
        nesting_level += 1

        start_time = time.time()
        result = method(*args, **kw)
        end_time = time.time()

        nesting_level -= 1
        log(f"End   [{method.__name__}]. Time elapsed: {end_time - start_time:0.2f} sec.")
        is_start = False

        return result

    return timed

In [16]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Training&Prediction by NN

## Functions

In [17]:
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [18]:
class RAdam(Optimizer):

    """RAdam optimizer.

    # Arguments
        lr: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        weight_decay: float >= 0. Weight decay for each param.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
        total_steps: int >= 0. Total number of training steps. Enable warmup by setting a positive value.
        warmup_proportion: 0 < warmup_proportion < 1. The proportion of increasing steps.
        min_lr: float >= 0. Minimum learning rate after warmup.
    # References
        - [Adam - A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)
        - [On The Variance Of The Adaptive Learning Rate And Beyond](https://arxiv.org/pdf/1908.03265v1.pdf)
    """

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., weight_decay=0., amsgrad=False,
                 total_steps=0, warmup_proportion=0.1, min_lr=0., **kwargs):
        super(RAdam, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.learning_rate = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.weight_decay = K.variable(weight_decay, name='weight_decay')
            self.total_steps = K.variable(total_steps, name='total_steps')
            self.warmup_proportion = K.variable(warmup_proportion, name='warmup_proportion')
            self.min_lr = K.variable(min_lr, name='min_lr')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.initial_weight_decay = weight_decay
        self.initial_total_steps = total_steps
        self.amsgrad = amsgrad

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.learning_rate

        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations, K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1

        if self.initial_total_steps > 0:
            warmup_steps = self.total_steps * self.warmup_proportion
            decay_steps = self.total_steps - warmup_steps
            lr = K.switch(
                t <= warmup_steps,
                lr * (t / warmup_steps),
                lr * (1.0 - K.minimum(t, decay_steps) / decay_steps),
            )

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='m_' + str(i)) for (i, p) in enumerate(params)]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='v_' + str(i)) for (i, p) in enumerate(params)]

        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='vhat_' + str(i)) for (i, p) in enumerate(params)]
        else:
            vhats = [K.zeros(1, name='vhat_' + str(i)) for i in range(len(params))]

        self.weights = [self.iterations] + ms + vs + vhats

        beta_1_t = K.pow(self.beta_1, t)
        beta_2_t = K.pow(self.beta_2, t)

        sma_inf = 2.0 / (1.0 - self.beta_2) - 1.0
        sma_t = sma_inf - 2.0 * t * beta_2_t / (1.0 - beta_2_t)

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)

            m_corr_t = m_t / (1.0 - beta_1_t)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                v_corr_t = K.sqrt(vhat_t / (1.0 - beta_2_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                v_corr_t = K.sqrt(v_t / (1.0 - beta_2_t) + self.epsilon)

            r_t = K.sqrt((sma_t - 4.0) / (sma_inf - 4.0) *
                         (sma_t - 2.0) / (sma_inf - 2.0) *
                         sma_inf / sma_t)

            p_t = K.switch(sma_t > 5, r_t * m_corr_t / v_corr_t, m_corr_t)

            if self.initial_weight_decay > 0:
                p_t += self.weight_decay * p

            p_t = p - lr * p_t

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {
            'learning_rate': float(K.get_value(self.learning_rate)),
            'beta_1': float(K.get_value(self.beta_1)),
            'beta_2': float(K.get_value(self.beta_2)),
            'decay': float(K.get_value(self.decay)),
            'weight_decay': float(K.get_value(self.weight_decay)),
            'epsilon': self.epsilon,
            'amsgrad': self.amsgrad,
            'total_steps': float(K.get_value(self.total_steps)),
            'warmup_proportion': float(K.get_value(self.warmup_proportion)),
            'min_lr': float(K.get_value(self.min_lr)),
        }
        base_config = super(RAdam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [19]:
class Lookahead(object):
    """Add the [Lookahead Optimizer](https://arxiv.org/abs/1907.08610) functionality for [keras](https://keras.io/).

    """
    def __init__(self, k=5, alpha=0.5):
        self.k = k
        self.alpha = alpha
        self.count = 0

    def inject(self, model):
        """Inject the Lookahead algorithm for the given model.
        The following code is modified from keras's _make_train_function method.
        See: https://github.com/keras-team/keras/blob/master/keras/engine/training.py#L497
        """
        if not hasattr(model, 'train_function'):
            raise RuntimeError('You must compile your model before using it.')

        model._check_trainable_weights_consistency()

        if model.train_function is None:
            inputs = (model._feed_inputs +
                      model._feed_targets +
                      model._feed_sample_weights)
            if model._uses_dynamic_learning_phase():
                inputs += [K.learning_phase()]
            fast_params = model._collected_trainable_weights

            with K.name_scope('training'):
                with K.name_scope(model.optimizer.__class__.__name__):
                    training_updates = model.optimizer.get_updates(
                        params=fast_params,
                        loss=model.total_loss)
                    slow_params = [K.variable(p) for p in fast_params]
                fast_updates = (model.updates +
                                training_updates +
                                model.metrics_updates)

                slow_updates, copy_updates = [], []
                for p, q in zip(fast_params, slow_params):
                    slow_updates.append(K.update(q, q + self.alpha * (p - q)))
                    copy_updates.append(K.update(p, q))

                # Gets loss and metrics. Updates weights at each call.
                fast_train_function = K.function(
                    inputs,
                    [model.total_loss] + model.metrics_tensors,
                    updates=fast_updates,
                    name='fast_train_function',
                    **model._function_kwargs)

                def F(inputs):
                    self.count += 1
                    R = fast_train_function(inputs)
                    if self.count % self.k == 0:
                        K.batch_get_value(slow_updates)
                        K.batch_get_value(copy_updates)
                    return R
                
                model.train_function = F

In [20]:
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(tf.keras.backend.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = tf.keras.backend.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [21]:
def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

In [22]:
class AdamW(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/4)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.learning_rate = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            # decoupled weight decay (2/4)
            self.wd = K.variable(weight_decay, name='weight_decay')
        self.epsilon = epsilon
        self.initial_decay = decay

#     @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd  # decoupled weight decay (3/4)

        lr = self.learning_rate
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            # decoupled weight decay (4/4)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - lr * wd * p

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.learning_rate)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'weight_decay': float(K.get_value(self.wd)),
                  'epsilon': self.epsilon}
        base_config = super(AdamW, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [23]:
class Position_Embedding(Layer): 
    def __init__(self, size=None, mode='sum', **kwargs):        
        self.size = size         
        self.mode = mode       
        super(Position_Embedding, self).__init__(**kwargs) 

    def call(self, x): 
        if (self.size == None) or (self.mode == 'sum'):            
            self.size = int(x.shape[-1])        
            batch_size, seq_len = K.shape(x)[0], K.shape(x)[1]        
            position_j = 1. / K.pow(10000., \
                2 * K.arange(self.size / 2, dtype='float32') / self.size)        
            position_j = K.expand_dims(position_j, 0)        
            position_i = K.cumsum(K.ones_like(x[:, :, 0]), 1)-1     
            position_i = K.expand_dims(position_i, 2)        
            position_ij = K.dot(position_i, position_j)        
            position_ij = K.concatenate([K.cos(position_ij), K.sin(position_ij)], 2) 
            if self.mode == 'sum': 
                return position_ij + x 
            elif self.mode == 'concat': 
                return K.concatenate([position_ij, x], 2) 

    def compute_output_shape(self, input_shape): 
        if self.mode == 'sum': 
            return input_shape 
        elif self.mode == 'concat': 
            return (input_shape[0], input_shape[1], input_shape[2]+self.size)

In [24]:
class FM(Layer): 
    def __init__(self, output_dim=30, activation="relu",**kwargs): 
        self.output_dim = output_dim 
        self.activate = activations.get(activation)
        super(FM, self).__init__(**kwargs) 
        
    def build(self, input_shape): 
        self.weight = self.add_weight(name='weight',shape=(input_shape[1], self.output_dim),initializer='glorot_uniform',trainable=True) 
        self.bias = self.add_weight(name='bias',shape=(self.output_dim,),initializer='zeros',trainable=True) 
        self.kernel = self.add_weight(name='kernel',shape=(input_shape[1], self.output_dim),initializer='glorot_uniform',trainable=True) 
        super(FM, self).build(input_shape) 
        
    def call(self, x):
        feature = K.dot(x,self.weight) + self.bias
        a = K.pow(K.dot(x,self.kernel), 2)
        b = K.dot(x, K.pow(self.kernel, 2))
        cross = K.mean(a-b, 1, keepdims=True)*0.5
        cross = K.repeat_elements(K.reshape(cross, (-1, 1)), self.output_dim, axis=-1) 
        return self.activate(feature + cross) 
    
    def compute_output_shape(self, input_shape): 
        return (input_shape[0], self.output_dim)

In [25]:
def baseline_model(inshape, outshape):
    # create two models
    input1 = Input(shape=inshape)
#     lstm_1 = LSTM(256)
    dense_1 = Dense(128, kernel_initializer='normal', activation='relu')(input1)
#     dense_1 = Dropout(0.5)(dense_1)
    dense_2 = Dense(64, kernel_initializer='normal', activation='relu')(dense_1)
#     dense_2 = Dropout(0.5)(dense_2)
    dense_3 = Dense(32, kernel_initializer='normal', activation='relu')(dense_2)
#     dense_3 = Dropout(0.5)(dense_3)
    out     = Dense(outshape,kernel_initializer='normal', activation='softmax')(dense_3)
    # Compile model
    model = Model(inputs=input1, outputs = out)
    model.compile(loss ='categorical_crossentropy', optimizer="adadelta", metrics=['accuracy'])  #logloss
    return model

In [26]:
def DenseNet(inshape, outshape):
    # create two models
    input1  = Input(shape=inshape)

    dense_1 = Dense(256, kernel_initializer='normal', activation='relu')(input1)
    #dense_1 = Dropout(0.6)(dense_1)
    dense_2 = Dense(128, kernel_initializer='normal', activation='relu')(dense_1)
    #dense_2 = Dropout(0.6)(dense_2)
    dense_2_x  = concatenate([dense_1,dense_2])
    dense_3 = Dense(64, kernel_initializer='normal', activation='relu')(dense_2_x)
    #dense_3 = Dropout(0.6)(dense_3)
    dense_3_x  = concatenate([dense_1,dense_2,dense_3])
    dense_4 = Dense(7, kernel_initializer='normal', activation='relu')(dense_3_x)
    #dense_4 = Dropout(0.6)(dense_4)
    dense_4_x  = concatenate([dense_1,dense_2,dense_3,dense_4])
    out     = Dense(outshape,kernel_initializer='normal', activation='softmax')(dense_4_x)
    # Compile model
    model = Model(inputs=input1, outputs = out)
    model.compile(loss ='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

In [27]:
def FMNet(inshape, outshape):
    # create two models
    input1  = Input(shape=inshape)
    #DNN_model_I
    dense_1 = Dense(100, kernel_initializer='normal', activation='tanh')(input1)
    dense_2 = Dense(150, kernel_initializer='normal', activation='tanh')(dense_1)
    dense_3 = Dense(150, kernel_initializer='normal', activation='tanh')(dense_2)
    dense_4 = Dense(100, kernel_initializer='normal', activation='tanh')(dense_3)
    dense_5 = Dense(64, kernel_initializer='normal', activation='tanh')(dense_4)
    #FM_model_II
    FM_1    = FM(200)(input1)
    FM_2    = FM(64)(FM_1)

    x       = concatenate([dense_5,FM_2])
    x_tmp   = Dense(32,kernel_initializer='normal', activation='softmax')(x)
    out     = Dense(outshape,kernel_initializer='normal', activation='softmax')(x_tmp)
    # Compile model
    model = Model(inputs=input1, outputs = out)
    model.compile(loss ='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    #model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

In [28]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init,
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     name='{}_b'.format(self.name),
                                     initializer='zero',
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [29]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

In [30]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [31]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, emded_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=emded_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=emded_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [32]:
class SoftAttention(object):
    """
    Layer to compute local inference between two encoded sentences a and b.
    """

    def __call__(self, inputs):
        a = inputs[0]
        b = inputs[1]

        attention = Lambda(self._attention,
                                        output_shape = self._attention_output_shape,
                                        arguments = None)(inputs)

        align_a = Lambda(self._soft_alignment,
                                     output_shape = self._soft_alignment_output_shape,
                                     arguments = None)([attention, a])
        align_b = Lambda(self._soft_alignment,
                                     output_shape = self._soft_alignment_output_shape,
                                     arguments = None)([attention, b])

        return align_a, align_b

    def _attention(self, inputs):
        """
        Compute the attention between elements of two sentences with the dot
        product.
        Args:
            inputs: A list containing two elements, one for the first sentence
                    and one for the second, both encoded by a BiLSTM.
        Returns:
            A tensor containing the dot product (attention weights between the
            elements of the two sentences).
        """
        attn_weights = K.batch_dot(x=inputs[0],
                                   y=K.permute_dimensions(inputs[1],
                                                          pattern=(0, 2, 1)))
        return K.permute_dimensions(attn_weights, (0, 2, 1))

    def _attention_output_shape(self, inputs):
        input_shape = inputs[0]
        embedding_size = input_shape[1]
        return (input_shape[0], embedding_size, embedding_size)

    def _soft_alignment(self, inputs):
        """
        Compute the soft alignment between the elements of two sentences.
        Args:
            inputs: A list of two elements, the first is a tensor of attention
                    weights, the second is the encoded sentence on which to
                    compute the alignments.
        Returns:
            A tensor containing the alignments.
        """
        attention = inputs[0]
        sentence = inputs[1]

        # Subtract the max. from the attention weights to avoid overflows.
        exp = K.exp(attention - K.max(attention, axis=-1, keepdims=True))
        exp_sum = K.sum(exp, axis=-1, keepdims=True)
        softmax = exp / exp_sum

        return K.batch_dot(softmax, sentence)

    def _soft_alignment_output_shape(self, inputs):
        attention_shape = inputs[0]
        sentence_shape = inputs[1]
        return (attention_shape[0], attention_shape[1], sentence_shape[2])


In [33]:
def esim():

    inp_a = Input(shape=(maxlen,))
    inp_b = Input(shape=(maxlen,))
    
    # embedding layer
    embedded_a = Embedding(max_features, embed_size, weights=[emb_matrix_dic["creative_id"]], trainable=False)(inp_a)
    embedded_b = Embedding(max_features, embed_size, weights=[emb_matrix_dic["ad_id"]], trainable=False)(inp_b)
    
    # Encoding Layer
    bilstm = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))
    
    encoded_a = bilstm(embedded_a)
    encoded_b = bilstm(embedded_b)
    
    # local inference layer 
    atten_a, atten_b = SoftAttention()([encoded_a, encoded_b])

    sub_a_atten = Lambda(lambda x: x[0]-x[1])([encoded_a, atten_a])
    sub_b_atten = Lambda(lambda x: x[0]-x[1])([encoded_b, atten_b])

    mul_a_atten = Lambda(lambda x: x[0]*x[1])([encoded_a, atten_a])
    mul_b_atten = Lambda(lambda x: x[0]*x[1])([encoded_b, atten_b])

    m_a = concatenate([encoded_a, atten_a, sub_a_atten, mul_a_atten])
    m_b = concatenate([encoded_b, atten_b, sub_b_atten, mul_b_atten])
    
    # Inference composition layer
    composition_a = Bidirectional(LSTM(
        128,
        return_sequences=True,
        dropout=0.1
    ))(m_a)

    avg_pool_a = GlobalAveragePooling1D()(composition_a)
    max_pool_a = GlobalMaxPooling1D()(composition_a)

    composition_b = Bidirectional(LSTM(
        128,
        return_sequences=True,
        dropout=0.1
    ))(m_b)

    avg_pool_b = GlobalAveragePooling1D()(composition_b)
    max_pool_b = GlobalMaxPooling1D()(composition_b)

    pooled = concatenate([avg_pool_a, max_pool_a, avg_pool_b, max_pool_b])
    pooled = Dropout(rate=0.1)(pooled)
    
    
    conc = Dense(256)(pooled)
    conc = BatchNormalization()(conc)
    conc = PReLU()(conc)
    conc = Dropout(0.2)(conc)

    conc = Dense(128)(conc)
    conc = BatchNormalization()(conc)
    # conc = PReLU()(conc)

    if num_classes == 1:
        outp = Dense(num_classes, activation="sigmoid")(conc)
    else:
        outp = Dense(num_classes, activation="softmax")(conc)

    model = Model(inputs=[inp_a, inp_b], outputs=outp)
    
    return model
    

In [34]:
def d_transformer():
    embed_dim = 256  # Embedding size for each token
    num_heads = 6  # Number of attention heads
    ff_dim = 128  # Hidden layer size in feed forward network inside transformer

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(20, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(2, activation="softmax")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)

## USE

In [35]:
feat_fname = sorted(os.listdir(UFEDIR))

In [36]:
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")
test_user = pd.read_csv(f"{DDIR}/test/user.csv")

In [ ]:
train_feat = pd.DataFrame()
test_feat = pd.DataFrame()
train_feat[UID] = train_user[UID]
test_feat[UID] = test_user[UID]

In [ ]:
train_feat.set_index(UID, inplace=True)
test_feat.set_index(UID, inplace=True)

In [ ]:
USE_IDSET = ID_SET[:-2]

In [ ]:
# word2vec
train_ftset_w2v = dict()
test_ftset_w2v = dict()
for cur_id in USE_IDSET:
    fname = f"w2v_avg_{cur_id}.pkl"
    print("current filename: ", fname)
    cur_df = pd.read_pickle(f"{UFEDIR}/{fname}")
    train_ftset_w2v[cur_id] = pd.merge(train_feat, cur_df, how="left", on=UID).values.reshape(-1, 1, 300)
    test_ftset_w2v[cur_id] = pd.merge(test_feat, cur_df, how="left", on=UID).values.reshape(-1, 1, 300)

In [ ]:
for cur_id in USE_IDSET:
    assert len(train_ftset_w2v[cur_id]) == len(train_feat)
    assert len(test_ftset_w2v[cur_id]) == len(test_feat)

In [ ]:
# doc2vec
train_ftset_d2v = dict()
test_ftset_d2v = dict()
for cur_id in USE_IDSET:
    fname = f"d2v_avg_{cur_id}.pkl"
    print("current filename: ", fname)
    cur_df = pd.read_pickle(f"{UFEDIR}/{fname}")
    train_ftset_d2v[cur_id] = pd.merge(train_feat, cur_df, how="left", on=UID).values.reshape(-1, 1, 300)
    test_ftset_d2v[cur_id] = pd.merge(test_feat, cur_df, how="left", on=UID).values.reshape(-1, 1, 300)

In [ ]:
for cur_id in USE_IDSET:
    assert len(train_ftset_d2v[cur_id]) == len(train_feat)
    assert len(test_ftset_d2v[cur_id]) == len(test_feat)

In [ ]:
# glove
train_ftset_glv = dict()
test_ftset_glv = dict()
for cur_id in USE_IDSET:
    fname = f"glove_avg_{cur_id}.pkl"
    print("current filename: ", fname)
    cur_df = pd.read_pickle(f"{UFEDIR}/{fname}")
    train_ftset_glv[cur_id] = pd.merge(train_feat, cur_df, how="left", on=UID).values.reshape(-1, 1, 300)
    test_ftset_glv[cur_id] = pd.merge(test_feat, cur_df, how="left", on=UID).values.reshape(-1, 1, 300)

In [ ]:
for cur_id in USE_IDSET:
    assert len(train_ftset_glv[cur_id]) == len(train_feat)
    assert len(test_ftset_glv[cur_id]) == len(test_feat)

In [ ]:
# tfidf svd
train_ftset_svd = dict()
test_ftset_svd = dict()
for cur_id in USE_IDSET:
    fname = f"tfidf_svd_{cur_id}.pkl"
    print("current filename: ", fname)
    cur_df = pd.read_pickle(f"{UFEDIR}/{fname}")
    train_ftset_svd[cur_id] = pd.merge(train_feat, cur_df, how="left", on=UID).values.reshape(-1, 1, 64)
    test_ftset_svd[cur_id] = pd.merge(test_feat, cur_df, how="left", on=UID).values.reshape(-1, 1, 64)

In [ ]:
for cur_id in USE_IDSET:
    assert len(train_ftset_svd[cur_id]) == len(train_feat)
    assert len(test_ftset_svd[cur_id]) == len(test_feat)

In [ ]:
# prediction and stats

In [ ]:
train_pemb = pd.DataFrame()
test_pemb = pd.DataFrame()
train_pemb[UID] = train_user[UID]
test_pemb[UID] = test_user[UID]

In [ ]:
for fname in feat_fname:
    if fname.startswith("meta_"):
        print("current filename: ", fname)
        gender_agg_pred = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_pemb = pd.merge(train_pemb, gender_agg_pred, how="left", on=UID)
        test_pemb = pd.merge(test_pemb, gender_agg_pred, how="left", on=UID)
        gender_agg_pred = None

In [ ]:
for fname in feat_fname:
    if fname.startswith("tfidf_count_emb_"):
        print("current filename: ", fname)
        cur_tce = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_pemb = pd.merge(train_pemb, cur_tce, how="left", on=UID)
        test_pemb = pd.merge(test_pemb, cur_tce, how="left", on=UID)

In [ ]:
# stats cate target encode
for fname in feat_fname:
    if fname.startswith("train_stats_catemlb"):
        print("current filename: ", fname)
        train_pemb = pd.merge(train_pemb, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
    elif fname.startswith("test_stats_catemlb"):
        print("current filename: ", fname)
        test_pemb = pd.merge(test_pemb, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

In [ ]:
# stats o1
for fname in feat_fname:
    if fname.startswith("train_stats_o1"):
        print("current filename: ", fname)
        train_pemb = pd.merge(train_pemb, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
    elif fname.startswith("test_stats_o1"):
        print("current filename: ", fname)
        test_pemb = pd.merge(test_pemb, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

In [ ]:
# stats getidxmax
for fname in feat_fname:
    if fname.startswith("train_stats_o2_getidxmax"):
        print("current filename: ", fname)
        train_pemb = pd.merge(train_pemb, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
    elif fname.startswith("test_stats_o2_getidxmax"):
        print("current filename: ", fname)
        test_pemb = pd.merge(test_pemb, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

In [ ]:
for fname in feat_fname:
    if fname.startswith("tfidf_svd_"):
        print("current filename: ", fname)
        cur_tfidf_svd = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_pemb = pd.merge(train_pemb, cur_tfidf_svd, how="left", on=UID)
        test_pemb = pd.merge(test_pemb, cur_tfidf_svd, how="left", on=UID)
        cur_tfidf_svd = None

In [ ]:
train_pemb.set_index(UID, inplace=True)
test_pemb.set_index(UID, inplace=True)

In [ ]:
set(train_pemb.columns == train_pemb.columns)

In [ ]:
# fillna and inf

In [ ]:
def nnaninf(val):
    return np.mean(val[(~np.isnan(val))&(~np.isinf(val))].values)

In [ ]:
def fillmean(df):
    for col in tqdm(df.columns):
        if df[col].count() < len(df):
            df[col] = df[col].replace([np.nan, np.inf], nnaninf(df[col]))
    # check
    for col in tqdm(df.columns):
        if df[col].count() < len(df):
            log(col)
    
    return df

In [ ]:
train_pemb = fillmean(train_pemb)
test_pemb = fillmean(test_pemb)

In [ ]:
print(train_pemb.values.max(), train_pemb.values.min())
print(test_pemb.values.max(), test_pemb.values.min())

In [ ]:
def transmms(tr_df, te_df):
    tol_df = pd.concat([tr_df, te_df])
    mms = MinMaxScaler()
    mms_tol_df = mms.fit_transform(tol_df)
    mms_tr_df = mms_tol_df[:len(tr_df)]
    mms_te_df = mms_tol_df[len(tr_df):]
    
    return mms_tr_df, mms_te_df

In [ ]:
mms_tr_df, mms_te_df = transmms(train_pemb, test_pemb)

In [ ]:
print(mms_tr_df.max(), mms_tr_df.min())
print(mms_te_df.max(), mms_te_df.min())

In [ ]:
mms_tr_df.shape

In [ ]:
mms_te_df.shape

In [37]:
def loadnprec(col, max_feature=None, maxlen=None, train_order=train_user[UID].values, test_order=test_user[UID].values):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    
    train_seq[col] = train_seq[col].apply(lambda x: " ".join(x))
    test_seq[col] = test_seq[col].apply(lambda x: " ".join(x))
    
    train_seq.sort_index(inplace=True)
    test_seq.sort_index(inplace=True)
    
    assert (train_seq.index.values != train_order).sum() == 0
    assert (test_seq.index.values != test_order).sum() == 0
    
    train_X = train_seq[col].values
    test_X = test_seq[col].values
    
    tol_X = np.concatenate([train_X, test_X])
    
    # Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features, filters="")
    tokenizer.fit_on_texts(list(tol_X))
    
    tol_X = tokenizer.texts_to_sequences(tol_X)

    # Pad the sentences 
    tol_X = pad_sequences(tol_X, maxlen=maxlen)
    
    train_X = tol_X[:len(train_seq)]
    test_X = tol_X[len(train_seq):]
    
    return train_X, test_X, tokenizer.word_index

In [38]:
def load_glove(col, word_index, max_features):
    
    EMBEDDING_FILE = (f"{UMDIR}/vectors/glove_{col}.model")
    k = Glove.load(EMBEDDING_FILE)
    
    embeddings_index = []
    for i in tqdm(k.dictionary.keys()):
        embeddings_index.append((i,k.word_vectors[k.dictionary[i]]))

    embeddings_index = dict(pd.DataFrame(embeddings_index).values)
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 

In [39]:
def load_w2v(col, word_index, max_features):    
    EMBEDDING_FILE = (f"{UMDIR}/vectors/w2v_{col}.model")
    embeddings_index = dict()
    model = gensim.models.Word2Vec.load(EMBEDDING_FILE)
    for word in model.wv.index2word:
        embeddings_index[word] = model.wv[word]
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [40]:
def loadnoh(col, max_feature=None, maxlen=None, train_order=train_user[UID].values, test_order=test_user[UID].values):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    
    train_seq[col] = train_seq[col].apply(lambda x: " ".join(x))
    test_seq[col] = test_seq[col].apply(lambda x: " ".join(x))
    
    train_seq.sort_index(inplace=True)
    test_seq.sort_index(inplace=True)
    
    assert (train_seq.index.values != train_order).sum() == 0
    assert (test_seq.index.values != test_order).sum() == 0
    
    train_X = train_seq[col].values
    test_X = test_seq[col].values
    
    tol_X = np.concatenate([train_X, test_X])
    
    tol_X = [one_hot(d, n=max_feature) for d in tol_X]

    # Pad the sentences 
    tol_X = pad_sequences(tol_X, maxlen=maxlen, padding='post')
    
    train_X = tol_X[:len(train_seq)]
    test_X = tol_X[len(train_seq):]
    
    return train_X, test_X

In [41]:
maxlen = 32
embed_size = 300
max_features = 50000

In [42]:
USE_IDSET = ["creative_id", "ad_id", "advertiser_id",]  #  "product_id", "industry"]

In [43]:
train_textprec_dic = dict()
test_textprec_dic = dict()
w2v_wordidx_dic = dict()
emb_matrix_dic = dict()

In [44]:
# for cur_id in tqdm(USE_IDSET):
#     train_textprec_dic[cur_id],  test_textprec_dic[cur_id] = loadnoh(cur_id, max_features, maxlen)

In [45]:
for cur_id in tqdm(USE_IDSET):
    train_textprec_dic[cur_id], test_textprec_dic[cur_id], w2v_wordidx_dic[cur_id] = loadnprec(cur_id, max_features, maxlen)
    emb_matrix_dic[cur_id] = load_w2v(cur_id, w2v_wordidx_dic[cur_id], max_features)

  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3331: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)
 33%|███▎      | 1/3 [04:13<08:26, 253.10s/it]/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3331: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)
 67%|██████▋   | 2/3 [08:19<04:11, 251.06s/it]/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3331: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non

In [49]:
# w2v_wordidx_dic

In [50]:
# emb_matrix_dic

In [46]:
# with open(f"{UDDIR}/imd/train_textprec_dic.pkl", "wb") as f:
#     pickle.dump(pickle.dumps(train_textprec_dic), f)

# with open(f"{UDDIR}/imd/test_textprec_dic.pkl", "wb") as f:
#     pickle.dump(pickle.dumps(test_textprec_dic), f)

# with open(f"{UDDIR}/imd/w2v_wordidx_dic.pkl", "wb") as f:
#     pickle.dump(pickle.dumps(w2v_wordidx_dic), f)

# with open(f"{UDDIR}/imd/emb_matrix_dic.pkl", "wb") as f:
#     pickle.dump(pickle.dumps(emb_matrix_dic), f)

In [ ]:
# with open(f"{UDDIR}/imd/train_textprec_dic.pkl", "rb") as f:
#     train_textprec_dic = pickle.loads(pickle.load(f))
    
# with open(f"{UDDIR}/imd/test_textprec_dic.pkl", "rb") as f:
#     test_textprec_dic = pickle.loads(pickle.load(f))

# with open(f"{UDDIR}/imd/w2v_wordidx_dic.pkl", "rb") as f:
#     w2v_wordidx_dic = pickle.loads(pickle.load(f))

# with open(f"{UDDIR}/imd/emb_matrix_dic.pkl", "rb") as f:
#     emb_matrix_dic = pickle.loads(pickle.load(f))

In [ ]:
for col in emb_matrix_dic:
    print(emb_matrix_dic[col].shape)

In [ ]:
train_textprec_dic["creative_id"].shape

In [ ]:
emb_matrix_dic["creative_id"]

In [ ]:
def cp_lstm_atten(inp, maxlen, max_features, embed_size, embedding_matrix):
    # Stage ctid
    x = Embedding(len(embedding_matrix), embed_size, weights=[embedding_matrix], trainable=False)(inp)
#     x = Position_Embedding()(x)
#     x = SpatialDropout1D(0.1)(x)
    y = Bidirectional(LSTM(128, return_sequences=True))(x)
#     y = Bidirectional(GRU(64, return_sequences=True))(x)
#     z = Conv1D(64, kernel_size = 1, kernel_initializer=initializers.he_uniform(seed=2020), activation = "tanh")(y)

#     atten_1 = Attention(maxlen)(x) # skip connect
#     atten_2 = Attention(maxlen)(y)
    avg_pool_1 = GlobalAveragePooling1D()(y)
    max_pool_1 = GlobalMaxPooling1D()(y)
#     max_pool1_1 = GlobalMaxPooling1D()(z)

#     convs = []
#     filter_sizes = [2, 3, 5, 8]
#     for fsz in filter_sizes:
#         l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
#         l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
#         l_pool = Flatten()(l_pool)
#         convs.append(l_pool)
#     text_cnn = concatenate(convs, axis=1)

    return avg_pool_1, max_pool_1

In [ ]:
def sim_lstm_atten(maxlen, num_classes, max_features, embed_size, embm_dic):
    
    w2v_inp = dict()
    
    for cur_id in USE_IDSET:
        w2v_inp[cur_id] = Input(shape=(maxlen, ))

    w2v_nn = dict()
        
    for cur_id in USE_IDSET:
        w2v_nn[cur_id] = list(cp_lstm_atten(w2v_inp[cur_id], maxlen, max_features, embed_size, embm_dic[cur_id]))
    
    inps = list(w2v_inp.values())
    nnls = list(np.array(list(w2v_nn.values())).flatten())
    
    # Stage prediction embedding
    inp_pemb = Input(shape=(mms_tr_df.shape[1], ))
    aux_1 = Dense(32)(inp_pemb)
#     aux_1 = Dense(32)(aux_1)
#     aux_1 = BatchNormalization()(aux_1)

#     all_nnls = nnls
#     all_inps = inps

    conc = concatenate(nnls+[aux_1])
    conc = Dense(256)(conc)
    conc = BatchNormalization()(conc)
    conc = PReLU()(conc)
    conc = Dropout(0.2)(conc)

    conc = Dense(128)(conc)
    conc = BatchNormalization()(conc)
    # conc = PReLU()(conc)

    if num_classes == 1:
        outp = Dense(num_classes, activation="sigmoid")(conc)
    else:
        outp = Dense(num_classes, activation="softmax")(conc)

    model = Model(inputs=inps+[inp_pemb], outputs=outp)

    return model

In [ ]:
DATA_SPLIT_SEED = 2020
cb = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),
      CyclicLR(base_lr=0.001, max_lr=0.002,
               step_size=300., mode='exp_range',
               gamma=0.99994)]

# for age
num_classes = 10
train_y = np_utils.to_categorical(train_user["age"]-1)


train_set =  list(train_textprec_dic.values())   # + [mms_tr_df, ]
test_set = list(test_textprec_dic.values())   # + [mms_te_df, ]

# train_set = list(train_textprec_dic.values())[:2]
# test_set = list(test_textprec_dic.values())[:2]

predicted_train_age = np.zeros(train_y.shape)
predicted_age = np.zeros((len(test_user),num_classes))
splits = list(KFold(n_splits=5, shuffle=True, random_state=DATA_SPLIT_SEED).split(train_set[0], train_y))

for idx, (train_idx, valid_idx) in enumerate(splits):
    if idx < 1:
        K.clear_session()

        X_train_set = list()
        X_val_set = list()

        for cur_train in train_set:
            X_train_set.append(cur_train[train_idx])
            X_val_set.append(cur_train[valid_idx])

        y_train = train_y[train_idx]
        y_val = train_y[valid_idx]


        model = sim_lstm_atten(maxlen, num_classes, max_features, embed_size, emb_matrix_dic)
#         model = esim()
        model.compile(loss='categorical_crossentropy', optimizer=RAdam(lr=0.001), metrics=['acc'])
        print("MODEL COMPLIE...")
        model.summary()

        model.fit(X_train_set, y_train, batch_size=2048, epochs=50, validation_data=(X_val_set, y_val), callbacks=cb, verbose=1)
        pred_val_y = model.predict(X_val_set, batch_size=2048, verbose=1, workers=20)
        pred_test_y = model.predict(test_set, batch_size=2048, verbose=1, workers=20)
        predicted_train_age[valid_idx] = pred_val_y
        predicted_age += pred_test_y / len(splits)
    

In [ ]:
pd.Series(np.argmax(predicted_train_age,axis=1)+1).value_counts()

In [ ]:
for idx, (train_idx, valid_idx) in enumerate(splits):
    print(accuracy_score(np.argmax(train_y[valid_idx],axis=1),np.argmax(predicted_train_age[valid_idx],axis=1)))

In [ ]:
maxlen=32

MODEL COMPLIE...
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 32, 300)      15000000    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 32, 300)      15000000    input_2[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 32, 300)      15000000    input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 32, 256)      439296      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 32, 256)      439296      embedding_2[0][0]                
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 32, 256)      439296      embedding_3[0][0]                
__________________________________________________________________________________________________
global_average_pooling1d_1 (Glo (None, 256)          0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
global_max_pooling1d_1 (GlobalM (None, 256)          0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
global_average_pooling1d_2 (Glo (None, 256)          0           bidirectional_2[0][0]            
__________________________________________________________________________________________________
global_max_pooling1d_2 (GlobalM (None, 256)          0           bidirectional_2[0][0]            
__________________________________________________________________________________________________
global_average_pooling1d_3 (Glo (None, 256)          0           bidirectional_3[0][0]            
__________________________________________________________________________________________________
global_max_pooling1d_3 (GlobalM (None, 256)          0           bidirectional_3[0][0]            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 1536)         0           global_average_pooling1d_1[0][0] 
                                                                 global_max_pooling1d_1[0][0]     
                                                                 global_average_pooling1d_2[0][0] 
                                                                 global_max_pooling1d_2[0][0]     
                                                                 global_average_pooling1d_3[0][0] 
                                                                 global_max_pooling1d_3[0][0]     
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 256)          393472      concatenate_1[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256)          1024        dense_1[0][0]                    
__________________________________________________________________________________________________
p_re_lu_1 (PReLU)               (None, 256)          256         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 256)          0           p_re_lu_1[0][0]                  
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 128)          32896       dropout_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 128)          512         dense_2[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 10)           1290        batch_normalization_2[0][0]      
==================================================================================================
Total params: 46,747,338
Trainable params: 1,746,570
Non-trainable params: 45,000,768
__________________________________________________________________________________________________
WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

Train on 720000 samples, validate on 180000 samples
Epoch 1/50
720000/720000 [==============================] - 348s 483us/step - loss: 1.7897 - acc: 0.3306 - val_loss: 1.5208 - val_acc: 0.3814
Epoch 2/50
720000/720000 [==============================] - 344s 478us/step - loss: 1.4606 - acc: 0.4042 - val_loss: 1.4580 - val_acc: 0.4038
Epoch 3/50
720000/720000 [==============================] - 344s 478us/step - loss: 1.4093 - acc: 0.4189 - val_loss: 1.4105 - val_acc: 0.4233
Epoch 4/50
720000/720000 [==============================] - 344s 477us/step - loss: 1.3683 - acc: 0.4349 - val_loss: 1.4058 - val_acc: 0.4198
Epoch 5/50
720000/720000 [==============================] - 426s 592us/step - loss: 1.3509 - acc: 0.4417 - val_loss: 1.3754 - val_acc: 0.4296
Epoch 6/50
720000/720000 [==============================] - 520s 722us/step - loss: 1.3230 - acc: 0.4527 - val_loss: 1.4390 - val_acc: 0.4139
Epoch 7/50
720000/720000 [==============================] - 381s 530us/step - loss: 1.3041 - acc: 0.4606 - val_loss: 1.3911 - val_acc: 0.4310
Epoch 8/50
720000/720000 [==============================] - 429s 596us/step - loss: 1.2869 - acc: 0.4668 - val_loss: 1.4427 - val_acc: 0.4267
180000/180000 [==============================] - 50s 280us/step
1000000/1000000 [==============================] - 199s 199us/step
                       
                       
0.42667777777777777
0.038933333333333334
0.03905
0.03961666666666667
0.039127777777777775

In [ ]:
DATA_SPLIT_SEED = 2020
cb = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),
      CyclicLR(base_lr=0.001, max_lr=0.002,
               step_size=300., mode='exp_range',
               gamma=0.99994)]

# for gender
num_classes = 1
train_y = (train_user["gender"]-1).values.reshape(-1, 1)

train_set =  list(train_textprec_dic.values()) + [mms_tr_df, ]
test_set = list(test_textprec_dic.values()) + [mms_te_df, ]


predicted_train_gender = np.zeros(train_y.shape)
predicted_gender = np.zeros((len(test_user),num_classes))
splits = list(KFold(n_splits=5, shuffle=True, random_state=DATA_SPLIT_SEED).split(train_set[0], train_y))

for idx, (train_idx, valid_idx) in enumerate(splits):
    if idx < 1:
        K.clear_session()

        X_train_set = list()
        X_val_set = list()

        for cur_train in train_set:
            X_train_set.append(cur_train[train_idx])
            X_val_set.append(cur_train[valid_idx])

        y_train = train_y[train_idx]
        y_val = train_y[valid_idx]

        model = sim_lstm_atten(maxlen, num_classes, max_features, embed_size, emb_matrix_dic)
        model.compile(loss='binary_crossentropy', optimizer=RAdam(lr=0.001), metrics=['acc'])
        print("MODEL COMPLIE...")
        model.summary()

        model.fit(X_train_set, y_train, batch_size=2048, epochs=50, validation_data=(X_val_set, y_val), callbacks=cb, verbose=1)
        pred_val_y = model.predict(X_val_set, batch_size=2048, verbose=1)
        pred_test_y = model.predict(test_set, batch_size=2048, verbose=1)
        predicted_train_gender[valid_idx] = pred_val_y
        predicted_gender += pred_test_y / len(splits)

In [ ]:
for idx, (train_idx, valid_idx) in enumerate(splits):
    print(accuracy_score((train_y[valid_idx] >= 0.5).astype(int),(predicted_train_gender[valid_idx] >= 0.5).astype(int)))

In [ ]:
0.42667777777777777 + 0.9337722222222222

In [ ]:
MODEL COMPLIE...
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 32, 300)      15000000    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 32, 300)      15000000    input_2[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 32, 300)      15000000    input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 32, 256)      439296      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 32, 256)      439296      embedding_2[0][0]                
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 32, 256)      439296      embedding_3[0][0]                
__________________________________________________________________________________________________
global_average_pooling1d_1 (Glo (None, 256)          0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
global_max_pooling1d_1 (GlobalM (None, 256)          0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
global_average_pooling1d_2 (Glo (None, 256)          0           bidirectional_2[0][0]            
__________________________________________________________________________________________________
global_max_pooling1d_2 (GlobalM (None, 256)          0           bidirectional_2[0][0]            
__________________________________________________________________________________________________
global_average_pooling1d_3 (Glo (None, 256)          0           bidirectional_3[0][0]            
__________________________________________________________________________________________________
global_max_pooling1d_3 (GlobalM (None, 256)          0           bidirectional_3[0][0]            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 1536)         0           global_average_pooling1d_1[0][0] 
                                                                 global_max_pooling1d_1[0][0]     
                                                                 global_average_pooling1d_2[0][0] 
                                                                 global_max_pooling1d_2[0][0]     
                                                                 global_average_pooling1d_3[0][0] 
                                                                 global_max_pooling1d_3[0][0]     
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 256)          393472      concatenate_1[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256)          1024        dense_1[0][0]                    
__________________________________________________________________________________________________
p_re_lu_1 (PReLU)               (None, 256)          256         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 256)          0           p_re_lu_1[0][0]                  
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 128)          32896       dropout_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 128)          512         dense_2[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 1)            129         batch_normalization_2[0][0]      
==================================================================================================
Total params: 46,746,177
Trainable params: 1,745,409
Non-trainable params: 45,000,768
__________________________________________________________________________________________________
Train on 720000 samples, validate on 180000 samples
Epoch 1/50
720000/720000 [==============================] - 347s 481us/step - loss: 0.2723 - acc: 0.8965 - val_loss: 0.2192 - val_acc: 0.9195
Epoch 2/50
720000/720000 [==============================] - 346s 481us/step - loss: 0.1948 - acc: 0.9292 - val_loss: 0.1824 - val_acc: 0.9338
Epoch 3/50
720000/720000 [==============================] - 357s 496us/step - loss: 0.1809 - acc: 0.9341 - val_loss: 0.1829 - val_acc: 0.9337
Epoch 4/50
720000/720000 [==============================] - 368s 510us/step - loss: 0.1726 - acc: 0.9376 - val_loss: 0.1775 - val_acc: 0.9360
Epoch 5/50
720000/720000 [==============================] - 349s 485us/step - loss: 0.1692 - acc: 0.9387 - val_loss: 0.1813 - val_acc: 0.9344
Epoch 6/50
720000/720000 [==============================] - 490s 681us/step - loss: 0.1621 - acc: 0.9415 - val_loss: 0.2092 - val_acc: 0.9275
Epoch 7/50
720000/720000 [==============================] - 420s 583us/step - loss: 0.1562 - acc: 0.9435 - val_loss: 0.1835 - val_acc: 0.9338
180000/180000 [==============================] - 35s 195us/step
1000000/1000000 [==============================] - 192s 192us/step
                       
0.9337722222222222
0.6710166666666667
0.6699944444444444
0.6692166666666667
0.6689833333333334

# Generate Prediction Result

In [ ]:
# model_lgb_multi_age = lgb.Booster(model_file=f"{UMDIR}/lgb_multi_age_20200511045531.model")

In [ ]:
# model_lgb_multi_gender = lgb.Booster(model_file=f"{UMDIR}/lgb_multi_gender_20200511034408.model")

In [ ]:
res = test_user[[UID]]

In [ ]:
res["predicted_age"] = np.argmax(predicted_age, axis=1)+1
# res["predicted_age"] = predicted_age
# res["predicted_age"] = [list(x).index(max(x))+1 for x in model_lgb_multi_age.predict(test_feat, num_iteration=model_lgb_multi_age.best_iteration)]

In [ ]:
res["predicted_gender"] = (predicted_gender >= 0.1).astype(int)+1
# res["predicted_gender"] = [list(x).index(max(x))+1 for x in model_lgb_multi_gender.predict(test_feat, num_iteration=model_lgb_multi_gender.best_iteration)]

In [ ]:
res_suffix = time.strftime("%Y%m%d%H%M%S", time.localtime(int(time.time())))
res.to_csv(f"{RESDIR}/res-{res_suffix}.csv", index=False)

In [ ]:
res["predicted_age"].value_counts()

In [ ]:
res["predicted_gender"].value_counts()

In [ ]:
res.shape

In [ ]:
tmp = pd.read_csv(f"{RESDIR}/res-20200605094137.csv")

In [ ]:
tmp["predicted_age"].value_counts()

In [ ]:
tmp["predicted_gender"].value_counts()

In [ ]:
res["predicted_gender"] = tmp["predicted_gender"]

In [ ]:
train_user.gender.value_counts()

In [ ]:
train_user.age.value_counts()

In [ ]:
gc.collect()

In [ ]:
tmp["predicted_age"]

# Cent result to COS

In [ ]:
from ti import session
ti_session = session.Session()

In [ ]:
inputs = ti_session.upload_data(path=f"{RESDIR}/res-20200515004850.csv", bucket="etveritas-1252104022", key_prefix=RESDIR)